# Munging the inaugural addresses


In [9]:
import urllib.request
# html = urllib.request.urlopen('http://avalon.law.yale.edu/subject_menus/inaug.asp', data = None)

In [10]:
html = urllib.request.urlopen('http://avalon.law.yale.edu/subject_menus/inaug.asp', data = None)

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')


In [12]:
soup.title

<title>Avalon Project - The Inaugural Addresses of the Presidents</title>

In [13]:
links = []
dates = []
pres = []
url_prefix = 'http://avalon.law.yale.edu/'
import re
for link in soup.find_all('a'):
    raw_link = link.get('href')
#     print(raw_link)
    if re.search('subject_menus', raw_link):
        continue
    elif re.search('about/purpose', raw_link):
        continue
    elif re.search('default', raw_link):
        continue
    elif re.match('http', raw_link):
        continue            
    else:
        links.append(url_prefix + raw_link[3:])
        dates.append(link.text)
        pres.append(link.find_previous('span').get_text())
    
print(links)
print(dates)
print(pres)

['http://avalon.law.yale.edu/18th_century/wash1.asp', 'http://avalon.law.yale.edu/18th_century/wash2.asp', 'http://avalon.law.yale.edu/18th_century/adams.asp', 'http://avalon.law.yale.edu/19th_century/jefinau1.asp', 'http://avalon.law.yale.edu/19th_century/jefinau2.asp', 'http://avalon.law.yale.edu/19th_century/madison1.asp', 'http://avalon.law.yale.edu/19th_century/madison2.asp', 'http://avalon.law.yale.edu/19th_century/monroe1.asp', 'http://avalon.law.yale.edu/19th_century/monroe2.asp', 'http://avalon.law.yale.edu/19th_century/qadams.asp', 'http://avalon.law.yale.edu/19th_century/jackson1.asp', 'http://avalon.law.yale.edu/19th_century/jackson2.asp', 'http://avalon.law.yale.edu/19th_century/vanburen.asp', 'http://avalon.law.yale.edu/19th_century/harrison.asp', 'http://avalon.law.yale.edu/19th_century/polk.asp', 'http://avalon.law.yale.edu/19th_century/taylor.asp', 'http://avalon.law.yale.edu/19th_century/pierce.asp', 'http://avalon.law.yale.edu/19th_century/buchanan.asp', 'http://aval

In [14]:
import pandas as pd

In [15]:
urls = pd.Series(links)
years = pd.Series(dates)
pres = pd.Series(pres)

inaugurals = pd.concat([urls, years, pres], axis = 1)
inaugurals.columns = ['urls', 'years', 'pres']
inaugurals.head()

,urls,years,pres
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson


In [16]:
# Another Way to get pres, but fails to account for multi-term presidents: 
pres_list = []
for span in soup.find_all('span'):
#     pres = span.get_text()
    pres = span.contents
#     pres_list.append(pres)
    pres_list = pres_list + pres
print(pres_list)

['George Washington', 'John Adams', 'Thomas Jefferson', 'James Madison', 'James Monroe', 'John Quincy Adams', 'Andrew Jackson', 'Martin Van Buren', 'William Henry Harrison', 'James Polk', 'Zachary Taylor', 'Franklin Pierce', 'James Buchanan', 'Abraham Lincoln', 'Ulysses S. Grant', 'Rutherford B. Hayes', 'James A. Garfield', 'Grover Cleveland', 'Benjamin Harrison', 'William McKinley', 'Theodore Roosevelt', 'William Howard Taft', 'Woodrow Wilson', 'Warren G. Harding', 'Calvin Coolidge', 'Herbert Hoover', 'Franklin D. Roosevelt', 'Harry S. Truman', 'Dwight D. Eisenhower', 'John F. Kennedy', 'Lyndon Baines Johnson', 'Richard Milhous Nixon', 'Jimmy Carter', 'Ronald Reagan', 'George Bush', 'William Clinton', 'George W. Bush', 'Barack Obama']


## Now get the texts
Here's where we go in and scrape the texts and add them to a column in our dataframe.

In [17]:
# A function to count the words
def wordcounter(text):
    if "This one was funny" in text:
        count = "N/A"
    else:
        words = text.split()
        count = 0
        for word in words:
            count += 1
    return count
        

In [48]:
# A function to strip the header from the scraped texts

def remove_str_head(text):
    try:
        start_pos = text.index("21st Century 2000 - ") + len("21st Century 2000 - ")
        try:
            end_pos = text.index("Inaugural Speeches Page")
        except:
            end_pos = text.index("Inauguration Speeches Page")
        print('SUCCESS')
        return(text[start_pos:end_pos])
    except:
        print('HEAD NOT STRIPPED')
        not_stripped_count += 1
        return text
        return not_stripped_count
    

In [46]:
# I don't use this function because the one above does a better job:
def remove_head(text):
    text_words_list = text.split()
    start_pos = text_words_list.index("2000")
    # start at '2000', which is the end of the heading on most pages
    print(start_pos)
    sequence = ["Inaugural", "Speeches", "Page"]
    for i in range(len(text_words_list)):
        if text_words_list[i:i+len(sequence)] == sequence:
            end_pos = i
    print(end_pos)
    text_words_list = text_words_list[start_pos:end_pos]
    text.join(text_words_list) # this is wrong
    return text
    print(text)




In [49]:
# A function to strip the texts
def format_text(url):
    html = urllib.request.urlopen(url, data = None)
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    text = remove_str_head(text)
    text = text.strip()
    text = text.replace('\n', ' ').replace('\r', ' ')
    print(text[:50])
    return text

In [50]:
import time
# Don't want to wear out our welcome on the yale server!
texts = []
fail_count = 0
not_stripped_count = 0
for url in inaugurals['urls']:
    print(url)
    try:
        scrape = format_text(url)
        # scrape = remove_str_head(scrape)
        # commented out because I put this in the previous step
        texts.append(scrape)
    except:
        texts.append("This one did not work:" + url)
        fail_count += 1
    time.sleep(1)

print("{} texts failed to load.".format(fail_count))
print("{} heads were not stripped off".format(not_stripped_count))


http://avalon.law.yale.edu/18th_century/wash1.asp
SUCCESS
First Inaugural Address of George Washington  THE 
http://avalon.law.yale.edu/18th_century/wash2.asp
SUCCESS
Second Inaugural Address of George Washington  THE
http://avalon.law.yale.edu/18th_century/adams.asp
SUCCESS
© 2008 Lillian Goldman Law Library 127 Wall Street
http://avalon.law.yale.edu/19th_century/jefinau1.asp
SUCCESS
The Avalon Project  at Yale Law School  Thomas Jef
http://avalon.law.yale.edu/19th_century/jefinau2.asp
SUCCESS
Thomas Jefferson Second Inaugural Address  Second 
http://avalon.law.yale.edu/19th_century/madison1.asp
SUCCESS
The Avalon Project  at Yale Law School James Madis
http://avalon.law.yale.edu/19th_century/madison2.asp
SUCCESS
The Avalon Project  at Yale Law School James Madis
http://avalon.law.yale.edu/19th_century/monroe1.asp
SUCCESS
First Inaugural Address of James Monroe  TUESDAY, 
http://avalon.law.yale.edu/19th_century/monroe2.asp
SUCCESS
Second Inaugural Address of James Monroe   MONDAY,
htt

In [21]:
# Remove the superfluous text before and after the addresses 
# I don't use this function because I built it into the scrape process above.

clean_texts = []
for text in texts:
    try:
        text = text.strip()
        clean_texts.append(text)
    except:
        clean_texts.append("N/A")

print(clean_texts[1])

Second Inaugural Address of George Washington  THE CITY OF PHILADELPHIA MONDAY, MARCH 4, 1793 Fellow Citizens:  I am again called upon by the voice of my country to execute the functions of its Chief Magistrate. When the occasion proper for it shall arrive, I shall endeavor to express the high sense I entertain of this distinguished honor, and of the confidence which has been reposed in me by the people of united America.  Previous to the execution of any official act of the President the Constitution requires an oath of office. This oath I am now about to take, and in your presence: That if it shall be found during my administration of the Government I have in any instance violated willingly or knowingly the injunctions thereof, I may (besides incurring constitutional punishment) be subject to the upbraidings of all who are now witnesses of the present solemn ceremony.


In [22]:
# split and count the words in the texts
# I don't use this function either.
texts = clean_texts

In [53]:
counts = []
for text in texts:
    counts.append(wordcounter(text))
    
print(counts)

[1443, 149, 2417, 1742, 2162, 1193, 1230, 3373, 4470, 2919, 1134, 1181, 5, 8434, 4805, 1096, 3335, 5, 3640, 711, 1132, 1346, 2483, 5, 1689, 2024, 4401, 3967, 2226, 992, 5438, 1709, 1536, 3336, 5, 3758, 1887, 1811, 1329, 557, 2276, 2452, 1647, 1350, 1490, 2115, 1797, 1232, 2438, 2564, 2319, 1601, 2176, 1595, 2073, 2417]


In [54]:
# now add the new list to the dataframe

inaugurals['texts'] = texts
inaugurals['wordcounts'] = counts

In [55]:
inaugurals



,urls,years,pres,texts,wordcounts
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington,First Inaugural Address of George Washington ...,1443
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington,Second Inaugural Address of George Washington ...,149
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams,© 2008 Lillian Goldman Law Library 127 Wall St...,2417
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson,The Avalon Project at Yale Law School Thomas...,1742
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson,Thomas Jefferson Second Inaugural Address Sec...,2162
5,http://avalon.law.yale.edu/19th_century/madiso...,1809,James Madison,The Avalon Project at Yale Law School James M...,1193
6,http://avalon.law.yale.edu/19th_century/madiso...,1813,James Madison,The Avalon Project at Yale Law School James M...,1230
7,http://avalon.law.yale.edu/19th_century/monroe...,1817,James Monroe,First Inaugural Address of James Monroe TUESD...,3373
8,http://avalon.law.yale.edu/19th_century/monroe...,1821,James Monroe,Second Inaugural Address of James Monroe MON...,4470
9,http://avalon.law.yale.edu/19th_century/qadams...,1825,John Quincy Adams,Inaugural Address of John Quincy Adams FRIDAY...,2919


In [56]:
for text in inaugurals['texts']:
    text.strip('\n')
    print(text[:100])
    print(type(text))

First Inaugural Address of George Washington  THE CITY OF NEW YORK THURSDAY, APRIL 30, 1789  Fellow-
<class 'str'>
Second Inaugural Address of George Washington  THE CITY OF PHILADELPHIA MONDAY, MARCH 4, 1793 Fellow
<class 'str'>
© 2008 Lillian Goldman Law Library 127 Wall Street, New Haven, CT 06511.  Avalon Statement of Purpos
<class 'str'>
The Avalon Project  at Yale Law School  Thomas Jefferson First Inaugural Address  First Inaugural Ad
<class 'str'>
Thomas Jefferson Second Inaugural Address  Second Inaugural Address  March 4, 1805  Proceeding, fell
<class 'str'>
The Avalon Project  at Yale Law School James Madison’s First Inaugural Address First Inaugural Addre
<class 'str'>
The Avalon Project  at Yale Law School James Madison’s First Inaugural Address First Inaugural Addre
<class 'str'>
First Inaugural Address of James Monroe  TUESDAY, MARCH 4, 1817 I should be destitute of feeling if 
<class 'str'>
Second Inaugural Address of James Monroe   MONDAY, MARCH 5, 1821 Fellow-Citizens

In [57]:
# Save the dataframe to csv
inaugurals.to_csv('data/inaugurals_may4.csv', index = False)

## Hand edits
I am trying my best to do all of the munging programatically, but a few things are preventing me.

- First, the avalon website is missing a few inaugural addresses, which I have to go fetch by hand.
- Secondly, although I tried to write a program to acquire the president's political parties from tables I found online, I could not figure a way to create a "parties" list to pair with the inaugural addresses. Some presidents of course served two terms, but most lists and tables I tried to scrape don't give multiple instances of the political parties to reflect multiple terms. In other words, I need FDR's "Democrat" to appear 4x in my list, so that when I zip the list together with my existing dataframe, each inaugural address for FDR gets one "Democrat" in the parties column. To make it even more complex, some presidents served without ever giving an inaugural address, etc., rendering one of the best online tables useless for my purposes. I think I could have figured this out eventually, but what's the point-- much easier to hand code.
- Finally, I wanted to add another column, which is New_Party or Incumbent_Party. I am sure I could make such a column in pandas, but it's easier to do this in excel because I know just what LOOKUP formula to use.

So in any case, my next step was to hand-code, and now I have to re-import the edited csv file back into a dataframe to continue my munging and then to move to analysis

In [65]:
import os
updated_df = pd.read_csv('./data/inaugurals_may4_incumb.csv')
updated_df

,urls,years,pres,texts,wordcounts,party,change
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington,First Inaugural Address of George Washington ...,1443,Federalist,New_Party
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington,Second Inaugural Address of George Washington ...,149,Federalist,Incumbent_Party
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams,© 2008 Lillian Goldman Law Library 127 Wall St...,2417,Democratic-Republican,New_Party
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson,The Avalon Project at Yale Law School Thomas...,1742,Democratic-Republican,Incumbent_Party
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson,Thomas Jefferson Second Inaugural Address Sec...,2162,Democratic-Republican,Incumbent_Party
5,http://avalon.law.yale.edu/19th_century/madiso...,1809,James Madison,The Avalon Project at Yale Law School James M...,1193,Democratic-Republican,Incumbent_Party
6,http://avalon.law.yale.edu/19th_century/madiso...,1813,James Madison,The Avalon Project at Yale Law School James M...,1230,Democratic-Republican,Incumbent_Party
7,http://avalon.law.yale.edu/19th_century/monroe...,1817,James Monroe,First Inaugural Address of James Monroe TUESD...,3373,Democratic-Republican,Incumbent_Party
8,http://avalon.law.yale.edu/19th_century/monroe...,1821,James Monroe,Second Inaugural Address of James Monroe MON...,4470,Democratic-Republican,Incumbent_Party
9,http://avalon.law.yale.edu/19th_century/qadams...,1825,John Quincy Adams,Inaugural Address of John Quincy Adams FRIDAY...,2919,Democratic-Republican,Incumbent_Party


## Recount the words
Some of the texts were not there in the previous df, so now to update the wordcounts:

In [73]:
# A function to count the words
def wordcounter(text):
    words = text.split()
    count = 0
    for word in words:
        count += 1
    return count

new_counts = []
for ind in range(len(updated_df)):
    count = wordcounter(updated_df['texts'][ind])
    # this attempt below (commented out) returned a warning about chained indexing
    # I did not have time to read and understand it, so I simply made a new colum with updated
    # wordcounts
    # updated_df['wordcounts'][ind] = count
    new_counts.append(count)
    print(count)
    
updated_df['new counts'] = new_counts
updated_df
    

1443
149
2417
1742
2162
1193
1230
3373
4470
2919
1134
1181
3826
8434
4805
1096
3335
2840
3640
711
1132
1346
2483
2973
1689
2024
4401
3967
2226
992
5438
1709
1536
3336
4055
3758
1887
1811
1329
557
2276
2452
1647
1350
1490
2115
1797
1232
2438
2564
2319
1601
2176
1595
2073
2417


,urls,years,pres,texts,wordcounts,party,change,new counts
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington,First Inaugural Address of George Washington ...,1443,Federalist,New_Party,1443
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington,Second Inaugural Address of George Washington ...,149,Federalist,Incumbent_Party,149
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams,© 2008 Lillian Goldman Law Library 127 Wall St...,2417,Democratic-Republican,New_Party,2417
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson,The Avalon Project at Yale Law School Thomas...,1742,Democratic-Republican,Incumbent_Party,1742
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson,Thomas Jefferson Second Inaugural Address Sec...,2162,Democratic-Republican,Incumbent_Party,2162
5,http://avalon.law.yale.edu/19th_century/madiso...,1809,James Madison,The Avalon Project at Yale Law School James M...,1193,Democratic-Republican,Incumbent_Party,1193
6,http://avalon.law.yale.edu/19th_century/madiso...,1813,James Madison,The Avalon Project at Yale Law School James M...,1230,Democratic-Republican,Incumbent_Party,1230
7,http://avalon.law.yale.edu/19th_century/monroe...,1817,James Monroe,First Inaugural Address of James Monroe TUESD...,3373,Democratic-Republican,Incumbent_Party,3373
8,http://avalon.law.yale.edu/19th_century/monroe...,1821,James Monroe,Second Inaugural Address of James Monroe MON...,4470,Democratic-Republican,Incumbent_Party,4470
9,http://avalon.law.yale.edu/19th_century/qadams...,1825,John Quincy Adams,Inaugural Address of John Quincy Adams FRIDAY...,2919,Democratic-Republican,Incumbent_Party,2919


## A bit more munging
I have only a few more things I want to do before analysis:

1. I want to pull out some of the text from the current "texts" column, specifically words like "Avalon," "Yale," and "Inaugural Address," and other metadata snippets that I failed to remove with my first cleaning attempts
2. I want to organize the df by date, because currently it's not quite chronological and that's bugging me. 
3. I want to start analysis!